In [4]:
# import numpy as np
# import pandas as pd
# import xgboost as xgb
# import optuna
# from sklearn.metrics import mean_squared_error
# from tqdm.auto import tqdm

# from src.dataset import get_datasets
# from src.feature import create_features, split
# # Adjust the import below to match your project structure.
# from src.benchmark import extend_by_predictions_and_samples

# # Prepare the data for checking candidate features.
# data = create_features(get_datasets()[0])
# data.index = pd.to_datetime(data.index)
# data['day'] = data.index.date

# # Define the candidate features.
# candidate_features = [
#     'Hydro',
#     'Pumped storage generation', 'Solar',
#     'Wind offshore', 'Wind onshore', 'temperature_2m', 'precipitation',
#     'wind_speed_100m', 'direct_radiation', 'hour', 'dayofweek', 'dayofyear'
# ]

# for i in range(3, 15, 2):
#     candidate_features.append(f'ma_{i}_days_pumped_storage_generation')
# candidate_features.append(f'ma_2_days_pumped_storage_generation')
# candidate_features.append(f'ma_14_days_pumped_storage_generation')

# def objective(trial):
#     # --------------------------------------------------
#     # 1. Feature Selection
#     # --------------------------------------------------
#     # Let Optuna decide whether to include each candidate feature.
#     selected_features = []
#     for feature in candidate_features:
#         use_feature = trial.suggest_categorical(feature, [True, False])
#         if use_feature and feature in data.columns:
#             selected_features.append(feature)
#     # If no feature was selected, fall back to using all candidate features.
#     if len(selected_features) == 0:
#         selected_features = candidate_features

#     # --------------------------------------------------
#     # 2. Hyperparameter Search Space for XGBoost
#     # --------------------------------------------------
#     # Sample early_stopping_rounds separately so it can be passed to fit.
#     params = {
#         # 'base_score': trial.suggest_float('base_score', 0.0, 1.0),
#         # 'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.5, log=True),
#         # 'max_depth': trial.suggest_int('max_depth', 3, 100),
#         # 'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
#         # 'gamma': trial.suggest_float('gamma', 0, 5.0),
#         # 'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
#         # 'subsample': trial.suggest_float('subsample', 0.5, 1.0),
#         # 'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1.0),
#         # 'max_delta_step': trial.suggest_int('max_delta_step', 0, 10),
#         # 'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 10, 250)
#     }

#     # --------------------------------------------------
#     # 3. Rolling Forecast Loop with Early Stopping
#     # --------------------------------------------------
#     merged_df, _ = get_datasets()
#     # The split function is assumed to return (train, eval, test, benchmark, …)
#     train_df, eval_df, test_df, benchmark_df, _, _, _ = split(merged_df)

#     # Combine available training data.
#     training_set = pd.concat([train_df, eval_df, test_df])
#     # Benchmarking set for rolling forecast.
#     benchmarking_set = benchmark_df

#     # Ensure the datasets are sorted by their datetime index.
#     training_set = training_set.sort_index()
#     benchmarking_set = benchmarking_set.sort_index()

#     WINDOW_SIZE = 24  # Forecast window size (24 hours)
#     rmse_scores = []

#     # Loop over forecast windows (starting at index 24 to ensure an initial training period).
#     for window_start in range(24, len(benchmarking_set) - WINDOW_SIZE, WINDOW_SIZE):
#         # The actual target values for the current forecast window.
#         y_actual = benchmarking_set.iloc[window_start: window_start + WINDOW_SIZE]["Price"]

#         # Extend the dataset with training data and all benchmark data up to the current window.
#         dataset_extended = pd.concat([training_set, benchmarking_set.iloc[:window_start]])
#         next_timestamp = dataset_extended.index[-1] + pd.DateOffset(hours=1)
#         if next_timestamp != y_actual.index[0]:
#             print(f"\nSkipping prediction for {next_timestamp} due to missing entries.\n"
#                   "--------------------------------------------------------------")
#             continue

#         # Optionally extend the dataset with predictions/samples.
#         dataset_extended_ps = extend_by_predictions_and_samples(
#             dataset_extended, dataset_extended.index[-1]
#         )

#         # Create features on the extended dataset.
#         dataset_extended_features = create_features(dataset_extended_ps)

#         # Check that the forecast window rows are present.
#         if not set(y_actual.index).issubset(dataset_extended_features.index):
#             print("Forecast window indices are missing in features, skipping.")
#             continue

#         # Split the extended dataset into training and forecast portions.
#         # Training data: all data except the last WINDOW_SIZE hours.
#         train_data = dataset_extended_features.iloc[:-WINDOW_SIZE]
#         # Create an evaluation set (the latter 20% of the training data) for early stopping.
#         split_idx = int(0.8 * len(train_data))
#         X_train = train_data.iloc[:split_idx][selected_features]
#         y_train = train_data.iloc[:split_idx]["Price"]
#         X_eval = train_data.iloc[split_idx:][selected_features]
#         y_eval = train_data.iloc[split_idx:]["Price"]

#         # The prediction set is the forecast window.
#         X_predict = dataset_extended_features.reindex(y_actual.index)[selected_features]
#         if X_predict.isnull().any().any():
#             print("Missing features in forecast window, skipping.")
#             continue

#         model = xgb.XGBRegressor(
#             **params,
#             objective='reg:squarederror',
#             eval_metric='rmse',
#             random_state=42,
#             n_jobs=-1
#         )

#         model.fit(
#             X_train, y_train,
#             eval_set=[(X_eval, y_eval)],
#             verbose=False
#         )

#         preds = model.predict(X_predict)
#         rmse = mean_squared_error(y_actual, preds)  ** 0.5
#         rmse_scores.append(rmse)

#     if len(rmse_scores) == 0:
#         return float('inf')
#     avg_rmse = np.mean(rmse_scores)
#     return avg_rmse

# # =============================================================================
# # Run the Optuna Study
# # =============================================================================
# n_trials = 300
# pbar = tqdm(total=n_trials, desc="Optuna Trials")

# def progress_callback(study, trial):
#     pbar.update(1)

# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=n_trials, n_jobs=-1, callbacks=[progress_callback])

# print("Best trial:")
# best_trial = study.best_trial
# print(f"  Average RMSE: {best_trial.value}")
# print("  Best hyperparameters:")
# for key, value in best_trial.params.items():
#     print(f"    {key}: {value}")


In [2]:
import optuna
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from prophet import Prophet
import pandas as pd
import numpy as np

from src.dataset import get_datasets
from src.feature import create_features, split
from src.benchmark import extend_by_predictions_and_samples

# =============================================================================
# Prepare the Data and Candidate Features
# =============================================================================

# Use an initial dataset to check available features.
initial_dataset = get_datasets()[0]
data = create_features(initial_dataset)
data.index = pd.to_datetime(data.index)
data['day'] = data.index.date

# Define candidate features.
candidate_features = [
    'Hydro',
    'Pumped storage generation',
    'Solar',
    'Wind offshore',
    'Wind onshore',
    'temperature_2m',
    'precipitation',
    'wind_speed_100m',
    'direct_radiation',
    'hour',
    'dayofweek',
    'dayofyear'
]

# Append moving-average features for various windows.
for i in range(3, 15, 2):
    candidate_features.append(f'ma_{i}_days_pumped_storage_generation')

# Append fixed moving-average features (only once).
candidate_features.append('ma_2_days_pumped_storage_generation')
candidate_features.append('ma_14_days_pumped_storage_generation')

# Remove duplicates (if any) while preserving order.
candidate_features = list(dict.fromkeys(candidate_features))


# =============================================================================
# Objective Function for Optuna
# =============================================================================
def objective(trial):
    # --------------------------------------------------
    # 1. Feature Selection
    # --------------------------------------------------
    # Let Optuna decide whether to include each candidate feature.
    selected_features = []
    for feature in candidate_features:
        use_feature = trial.suggest_categorical(feature, [True, False])
        if use_feature and feature in data.columns:
            selected_features.append(feature)
    # If no feature was selected, fall back to using all candidate features.
    if len(selected_features) == 0:
        selected_features = candidate_features
    # Remove any duplicates.
    selected_features = list(dict.fromkeys(selected_features))
    
    # --------------------------------------------------
    # 2. Hyperparameter Search Space for Prophet (Optional)
    # --------------------------------------------------
    # Uncomment the lines below if you want to tune these parameters.
    # changepoint_prior_scale = trial.suggest_float('changepoint_prior_scale', 0.001, 0.5, log=True)
    # seasonality_prior_scale = trial.suggest_float('seasonality_prior_scale', 1.0, 20.0, log=True)
    # holidays_prior_scale    = trial.suggest_float('holidays_prior_scale', 1.0, 20.0, log=True)
    # seasonality_mode        = trial.suggest_categorical('seasonality_mode', ['additive', 'multiplicative'])
    
    # --------------------------------------------------
    # 3. Rolling Forecast Loop with Early Stopping
    # --------------------------------------------------
    merged_df, _ = get_datasets()
    # The split function is assumed to return (train, eval, test, benchmark, …)
    train_df, eval_df, test_df, benchmark_df, _, _, _ = split(merged_df)

    # Combine available training data.
    training_set = pd.concat([train_df, eval_df, test_df])
    benchmarking_set = benchmark_df

    # Ensure the datasets are sorted by their datetime index.
    training_set = training_set.sort_index()
    benchmarking_set = benchmarking_set.sort_index()

    WINDOW_SIZE = 24  # Forecast window size (24 hours)
    rmse_scores = []

    # Loop over forecast windows (starting at index 24 to ensure an initial training period).
    for window_start in range(24, len(benchmarking_set) - WINDOW_SIZE, WINDOW_SIZE):
        # The actual target values for the current forecast window.
        y_actual = benchmarking_set.iloc[window_start: window_start + WINDOW_SIZE]["Price"]

        # Extend the dataset with training data and benchmark data up to the current window.
        dataset_extended = pd.concat([training_set, benchmarking_set.iloc[:window_start]])
        next_timestamp = dataset_extended.index[-1] + pd.DateOffset(hours=1)
        if next_timestamp != y_actual.index[0]:
            print(f"\nSkipping prediction for {next_timestamp} due to missing entries.\n"
                  "--------------------------------------------------------------")
            continue

        # Optionally extend the dataset with predictions/samples.
        dataset_extended_ps = extend_by_predictions_and_samples(
            dataset_extended, dataset_extended.index[-1]
        )

        # Create features on the extended dataset.
        dataset_extended_features = create_features(dataset_extended_ps)

        # Ensure forecast window rows are present in the features.
        if not set(y_actual.index).issubset(dataset_extended_features.index):
            print("Forecast window indices are missing in features, skipping.")
            continue

        # Split for training (all but last 24 hours) and prediction (the next 24 hours)
        X_train_df = dataset_extended_features.iloc[:-WINDOW_SIZE]
        try:
            # Use .loc to select the forecast window rows.
            X_predict_df = dataset_extended_features.loc[y_actual.index, selected_features]
        except KeyError as e:
            print("Error selecting forecast features:", e)
            continue

        if X_predict_df.isnull().any().any():
            print("Missing features in forecast window, skipping.")
            continue

        # Prepare Prophet-compatible training dataframe.
        X_train_prophet = (
            X_train_df[["Price"] + selected_features]
            .rename(columns={"Price": "y"})
            .reset_index()
            .rename(columns={"index": "ds"})
            .dropna()
        )
        prophet_X_predict = X_predict_df.reset_index().rename(columns={"index": "ds"})

        # Initialize and fit the Prophet model.
        model = Prophet(
            # Uncomment the lines below if you want to pass tuned hyperparameters:
            # changepoint_prior_scale=changepoint_prior_scale,
            # seasonality_prior_scale=seasonality_prior_scale,
            # holidays_prior_scale=holidays_prior_scale,
            # seasonality_mode=seasonality_mode
        )
       
        # Add selected regressors.
        for reg in selected_features:
            model.add_regressor(reg)

        model.fit(X_train_prophet)
       
        # Generate a forecast.
        forecast = model.predict(prophet_X_predict)
        prophet_forecast = forecast["yhat"]
        rmse = mean_squared_error(y_actual, prophet_forecast) ** 0.5
        rmse_scores.append(rmse)

    if len(rmse_scores) == 0:
        return float('inf')
    avg_rmse = np.mean(rmse_scores)
    return avg_rmse


# =============================================================================
# Run the Optuna Study
# =============================================================================
n_trials = 100
pbar = tqdm(total=n_trials, desc="Optuna Trials")

def progress_callback(study, trial):
    pbar.update(1)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=n_trials, n_jobs=-1, callbacks=[progress_callback])
pbar.close()

print("Best trial:")
best_trial = study.best_trial
print(f"  Average RMSE: {best_trial.value}")
print("  Best hyperparameters:")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")


Optuna Trials:   0%|          | 0/10 [00:00<?, ?it/s]

[I 2025-02-09 21:15:09,307] A new study created in memory with name: no-name-93fde0cf-9498-45d1-a520-308e9a044638
21:15:41 - cmdstanpy - INFO - Chain [1] start processing
21:15:42 - cmdstanpy - INFO - Chain [1] start processing
21:15:43 - cmdstanpy - INFO - Chain [1] start processing
21:15:43 - cmdstanpy - INFO - Chain [1] start processing
21:15:43 - cmdstanpy - INFO - Chain [1] start processing
21:15:43 - cmdstanpy - INFO - Chain [1] start processing
21:15:45 - cmdstanpy - INFO - Chain [1] start processing
21:15:45 - cmdstanpy - INFO - Chain [1] start processing
21:15:46 - cmdstanpy - INFO - Chain [1] start processing
21:15:46 - cmdstanpy - INFO - Chain [1] start processing
21:16:43 - cmdstanpy - INFO - Chain [1] done processing
21:16:46 - cmdstanpy - INFO - Chain [1] done processing
21:16:46 - cmdstanpy - INFO - Chain [1] done processing
21:16:58 - cmdstanpy - INFO - Chain [1] done processing
21:17:03 - cmdstanpy - INFO - Chain [1] start processing
21:17:06 - cmdstanpy - INFO - Chain

KeyboardInterrupt: 

21:24:01 - cmdstanpy - INFO - Chain [1] start processing
21:24:02 - cmdstanpy - INFO - Chain [1] done processing
21:24:03 - cmdstanpy - INFO - Chain [1] done processing
21:24:04 - cmdstanpy - INFO - Chain [1] start processing
21:24:09 - cmdstanpy - INFO - Chain [1] start processing
21:24:13 - cmdstanpy - INFO - Chain [1] start processing
21:24:14 - cmdstanpy - INFO - Chain [1] done processing
21:24:17 - cmdstanpy - INFO - Chain [1] done processing
21:24:22 - cmdstanpy - INFO - Chain [1] start processing
21:24:26 - cmdstanpy - INFO - Chain [1] done processing
21:24:26 - cmdstanpy - INFO - Chain [1] start processing
21:24:42 - cmdstanpy - INFO - Chain [1] start processing
21:24:47 - cmdstanpy - INFO - Chain [1] start processing
21:24:49 - cmdstanpy - INFO - Chain [1] start processing
21:24:50 - cmdstanpy - INFO - Chain [1] start processing
21:25:08 - cmdstanpy - INFO - Chain [1] done processing
21:25:12 - cmdstanpy - INFO - Chain [1] done processing
21:25:17 - cmdstanpy - INFO - Chain [1